# 🛠️ **Módulo 8: Projetos Práticos - Colocando a Mão na Massa**

> *Agora vamos construir projetos reais e funcionais com RAG!*

---

## **Aula 8.1: Projeto 1 - Chatbot para E-commerce**

---

### **Tá, mas o que vamos construir?**

Vamos criar um **chatbot inteligente para e-commerce** que:
- 🛍️ **Conhece todos os produtos**
- �� **Atende clientes 24/7**
- 🧠 **Lembra das preferências**
- 🔍 **Busca produtos inteligentemente**
- 💳 **Ajuda com compras**

É como ter um **vendedor digital super inteligente** que nunca dorme e conhece cada cliente! 🛒

**🖼️ Sugestão de imagem**: Um chatbot de e-commerce atendendo clientes com produtos ao redor

### **Por que este projeto é importante?**

**Problema real**: E-commerces perdem vendas por falta de atendimento
**Solução**: Chatbot que atende instantaneamente e converte vendas

É como a diferença entre **uma loja fechada** e **uma loja com vendedor 24h**! 🏪

---

### **Setup Inicial - Preparando o Terreno**

**⚠️ IMPORTANTE**: Se você não executou o notebook `00_setup_colab.ipynb` primeiro, execute a célula abaixo para instalar as dependências.

In [ ]:
# 🚀 SETUP GRATUITO PARA COLAB
# Execute esta célula primeiro para configurar o ambiente!

# Instalando dependências (execute apenas se necessário)
!pip install langchain>=0.1.0
!pip install langchain-community>=0.0.10
!pip install langchain-core>=0.1.0
!pip install python-dotenv>=1.0.0
!pip install huggingface_hub>=0.19.0
!pip install sentence-transformers>=2.2.0
!pip install chromadb>=0.4.0
!pip install faiss-cpu>=1.7.0
!pip install numpy>=1.24.0
!pip install pandas>=2.0.0

print("✅ Dependências instaladas com sucesso!")
print("��️ Pronto para criar o chatbot de e-commerce!")

In [ ]:
# 🛍️ IMPORTAÇÕES PARA O PROJETO E-COMMERCE
import os
import json
from dotenv import load_dotenv

# LangChain
from langchain_community.llms import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document

# Utilitários
import pandas as pd

print("✅ Bibliotecas importadas com sucesso!")
print("��️ Pronto para criar o vendedor digital!")

### **Passo 1 - Criando a Base de Produtos**

Primeiro, vamos criar uma **base de produtos realista** para nossa loja virtual. É como criar um catálogo digital! 📚

In [ ]:
# �� PASSO 1: CRIANDO A BASE DE PRODUTOS

# Base de produtos da TechStore (nossa loja virtual)
produtos_ecommerce = """
CATÁLOGO DE PRODUTOS - TECHSTORE

SMARTPHONES:

iPhone 15 Pro Max
- Preço: R$ 9.999
- Categoria: Smartphone
- Marca: Apple
- Características: Tela 6.7", Chip A17 Pro, Câmera tripla 48MP
- Cores: Titânio Natural, Titânio Azul, Titânio Branco, Titânio Preto
- Capacidades: 256GB, 512GB, 1TB
- Garantia: 1 ano
- Status: Em estoque

Samsung Galaxy S24 Ultra
- Preço: R$ 8.999
- Categoria: Smartphone
- Marca: Samsung
- Características: Tela 6.8", Snapdragon 8 Gen 3, Câmera quádrupla 200MP
- Cores: Titânio Cinza, Titânio Preto, Titânio Violeta, Titânio Amarelo
- Capacidades: 256GB, 512GB, 1TB
- Garantia: 1 ano
- Status: Em estoque

Google Pixel 8 Pro
- Preço: R$ 7.499
- Categoria: Smartphone
- Marca: Google
- Características: Tela 6.7", Tensor G3, Câmera tripla 50MP
- Cores: Obsidian, Porcelain, Bay
- Capacidades: 128GB, 256GB, 512GB
- Garantia: 1 ano
- Status: Em estoque

NOTEBOOKS:

MacBook Pro 14" M3 Pro
- Preço: R$ 18.999
- Categoria: Notebook
- Marca: Apple
- Características: Chip M3 Pro, 14", 18GB RAM, 512GB SSD
- Cores: Space Gray, Silver
- Garantia: 1 ano
- Status: Em estoque

Dell XPS 13 Plus
- Preço: R$ 12.999
- Categoria: Notebook
- Marca: Dell
- Características: Intel i7-1360P, 13.4", 16GB RAM, 512GB SSD
- Cores: Platinum Silver, Graphite
- Garantia: 1 ano
- Status: Em estoque

ACESSÓRIOS:

AirPods Pro 2ª Geração
- Preço: R$ 2.499
- Categoria: Fones de Ouvido
- Marca: Apple
- Características: Cancelamento de ruído ativo, Áudio espacial
- Cores: Branco
- Garantia: 1 ano
- Status: Em estoque

Apple Watch Series 9
- Preço: R$ 3.999
- Categoria: Smartwatch
- Marca: Apple
- Características: Tela 45mm, Chip S9, Monitor cardíaco
- Cores: Midnight, Starlight, Pink, Blue, Red
- Garantia: 1 ano
- Status: Em estoque

POLÍTICAS DA LOJA:

ENTREGA:
- Frete grátis para compras acima de R$ 500
- Entrega em 1-3 dias úteis
- Rastreamento em tempo real

PAGAMENTO:
- Cartão de crédito (até 12x)
- PIX (5% de desconto)
- Boleto bancário

GARANTIA:
- Todos os produtos têm garantia de fábrica
- Suporte técnico especializado
- Troca em até 7 dias

ATENDIMENTO:
- Chat 24/7
- WhatsApp: (11) 99999-9999
- Email: atendimento@techstore.com
"""

# Salvando base de produtos
with open('produtos_techstore.txt', 'w', encoding='utf-8') as file:
    file.write(produtos_ecommerce)

print("🛍️ Base de produtos criada!")
print("�� Contém smartphones, notebooks e acessórios da TechStore")
print("�� Arquivo 'produtos_techstore.txt' salvo com sucesso!")

### **Passo 2 - Criando o Chatbot de E-commerce**

Agora vamos criar o **vendedor digital super inteligente** que conhece todos os produtos! 🤖

In [ ]:
# �� PASSO 2: CRIANDO O CHATBOT DE E-COMMERCE

# Configurando o LLM (nosso vendedor inteligente)
def get_llm_colab():
    """Retorna o melhor LLM disponível no Colab"""
    
    # Tentativa 1: OpenAI (se tiver API key)
    try:
        from langchain_openai import ChatOpenAI
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            return ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.7,
                api_key=api_key
            )
    except:
        pass
    
    # Tentativa 2: Hugging Face (gratuito)
    try:
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        if token:
            return HuggingFaceHub(
                repo_id="google/flan-t5-base",
                model_kwargs={"temperature": 0.7, "max_length": 512}
            )
    except:
        pass
    
    # Fallback: LLM simples
    return HuggingFaceHub(
        repo_id="google/flan-t5-base",
        model_kwargs={"temperature": 0.7, "max_length": 512}
    )

# Configurando o LLM
llm = get_llm_colab()

print(f"🤖 Vendedor digital configurado: {type(llm).__name__}")
print("🛍️ Pronto para atender clientes!")

In [ ]:
# 🧠 CRIANDO O SISTEMA RAG PARA E-COMMERCE

# Carregando e processando base de produtos
from langchain.document_loaders import TextLoader

loader = TextLoader('produtos_techstore.txt', encoding='utf-8')
documentos = loader.load()

print(f"📄 Base de produtos carregada: {len(documentos)} documento")

# Dividindo em pedaços menores
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
textos_divididos = text_splitter.split_documents(documentos)

print(f"�� Produtos divididos em {len(textos_divididos)} seções")

# Criando embeddings e vector store
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma.from_documents(
    documents=textos_divididos,
    embedding=embeddings,
    collection_name="produtos_techstore"
)

print("🧠 Vector store de produtos criado!")
print("🔍 Sistema de busca inteligente ativo!")

In [ ]:
# ��️ CRIANDO O CHATBOT COM MEMÓRIA

# Criando memory para lembrar das preferências do cliente
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Criando chatbot com RAG
chatbot_ecommerce = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True,
    verbose=True
)

print("🤖 Chatbot de E-commerce criado!")
print("🛍️ Vendedor digital pronto para atender!")
print("🧠 Com memória para lembrar das preferências!")

### **Teste Rápido - Nosso Vendedor Digital em Ação!**

Vamos testar nosso chatbot com algumas perguntas típicas de clientes:

In [ ]:
# 🧪 TESTE RÁPIDO: NOSSO VENDEDOR DIGITAL EM AÇÃO!

print("🛍️ TESTANDO CHATBOT DE E-COMMERCE")
print("=" * 60)

# Simulando conversas de clientes
conversas_teste = [
    "Olá! Estou procurando um smartphone bom para fotos.",
    "Qual a diferença entre iPhone 15 Pro Max e Samsung Galaxy S24 Ultra?",
    "Quanto custa o iPhone 15 Pro Max?",
    "Vocês têm frete grátis?",
    "Lembra do que eu estava procurando?",
    "Quais são as formas de pagamento?",
    "Preciso de um notebook para trabalho. O que você recomenda?"
]

for i, mensagem in enumerate(conversas_teste, 1):
    print(f"\n�� Cliente {i}: {mensagem}")
    print("-" * 50)
    
    try:
        # Executando chatbot
        resultado = chatbot_ecommerce({"question": mensagem})
        
        print(f"🤖 Vendedor: {resultado['answer'][:300]}...")
        
        if resultado['source_documents']:
            print(f"📄 Consultou {len(resultado['source_documents'])} produtos")
        
    except Exception as e:
        print(f"❌ Erro: {e}")
    
    print("-" * 50)

print("\n🎉 Chatbot funcionando perfeitamente!")
print("🛍️ Vendedor digital pronto para converter vendas!")

## **Aula 8.2: Projeto 2 - Assistente de Estudos**

---

### **O que vamos construir?**

Vamos criar um **assistente de estudos inteligente** que:
- 📚 **Analisa documentos acadêmicos**
- 📝 **Gera resumos automáticos**
- 🎴 **Cria flashcards**
- ❓ **Sistema de perguntas e respostas**
- 📊 **Acompanha o progresso**

É como ter um **professor particular 24h** que nunca se cansa! 👨‍🏫

**🖼️ Sugestão de imagem**: Um professor digital ajudando um estudante com livros ao redor

### **Por que este projeto é útil?**

**Problema real**: Estudantes têm dificuldade para organizar e revisar material
**Solução**: Assistente que automatiza tarefas de estudo

É como a diferença entre **estudar sozinho** e **ter um tutor personalizado**! 📖

In [ ]:
# 📚 CRIANDO BASE DE CONHECIMENTO ACADÊMICA

# Material de estudo sobre Inteligência Artificial
material_estudo = """
INTRODUÇÃO À INTELIGÊNCIA ARTIFICIAL

DEFINIÇÃO E CONCEITOS BÁSICOS:

Inteligência Artificial (IA) é um campo da computação que busca criar sistemas
capazes de realizar tarefas que normalmente requerem inteligência humana.
A IA combina ciência da computação, matemática, psicologia e outras disciplinas.

HISTÓRICO DA IA:

A IA nasceu na década de 1950, com o teste de Turing proposto por Alan Turing.
O termo "Inteligência Artificial" foi cunhado em 1956 na Conferência de Dartmouth.
Desde então, a IA passou por vários "invernos" e "primaveras" de desenvolvimento.

TIPOS DE INTELIGÊNCIA ARTIFICIAL:

1. IA Fraca (Narrow AI):
   - Especializada em uma tarefa específica
   - Exemplos: Siri, Alexa, sistemas de recomendação
   - Não possui consciência ou inteligência geral

2. IA Forte (AGI - Artificial General Intelligence):
   - Inteligência geral como humanos
   - Pode realizar qualquer tarefa intelectual
   - Ainda não foi alcançada

3. Superinteligência:
   - Inteligência superior à humana
   - Conceito teórico e controverso

MACHINE LEARNING:

Machine Learning é um subcampo da IA que permite aos computadores aprender
sem serem explicitamente programados. Os algoritmos identificam padrões
nos dados para fazer previsões ou tomar decisões.

TIPOS DE MACHINE LEARNING:

1. Aprendizado Supervisionado:
   - Usa dados rotulados para treinar
   - Exemplos: classificação, regressão
   - Algoritmos: Random Forest, SVM, Redes Neurais

2. Aprendizado Não Supervisionado:
   - Encontra padrões em dados não rotulados
   - Exemplos: clustering, redução de dimensionalidade
   - Algoritmos: K-means, PCA, Autoencoders

3. Aprendizado por Reforço:
   - Aprende através de tentativa e erro
   - Usa recompensas e punições
   - Exemplos: jogos, robótica, carros autônomos

DEEP LEARNING:

Deep Learning é um subcampo do Machine Learning que usa redes neurais
com múltiplas camadas para aprender representações complexas dos dados.

CARACTERÍSTICAS:
- Múltiplas camadas de neurônios
- Aprendizado de features automático
- Requer grandes quantidades de dados
- Poder computacional significativo

APLICAÇÕES PRÁTICAS:

1. Visão Computacional:
   - Reconhecimento de imagens
   - Detecção de objetos
   - Processamento de vídeo

2. Processamento de Linguagem Natural:
   - Tradução automática
   - Análise de sentimentos
   - Chatbots inteligentes

3. Sistemas de Recomendação:
   - Netflix, Spotify, Amazon
   - Personalização de conteúdo
   - Marketing direcionado

4. Medicina e Saúde:
   - Diagnóstico por imagem
   - Descoberta de medicamentos
   - Medicina personalizada

DESAFIOS E LIMITAÇÕES:

1. Viés nos Dados:
   - Dados podem refletir preconceitos
   - Importante para justiça e equidade

2. Interpretabilidade:
   - Modelos complexos são difíceis de explicar
   - Importante para confiança e responsabilidade

3. Privacidade:
   - Coleta e uso de dados pessoais
   - Necessidade de proteção

4. Segurança:
   - Vulnerabilidades em sistemas de IA
   - Ataques adversariais

FUTURO DA IA:

O futuro da IA inclui:
- IA mais ética e responsável
- Automação de mais tarefas
- Integração com IoT e robótica
- Desenvolvimento de AGI (controverso)
- Regulamentações e políticas

CONCEITOS IMPORTANTES:

Overfitting: Modelo que memoriza dados de treino
Underfitting: Modelo muito simples para os dados
Cross-validation: Técnica para avaliar modelos
Feature Engineering: Criação de features relevantes
Hyperparameter Tuning: Otimização de parâmetros
"""

# Salvando material de estudo
with open('material_ia.txt', 'w', encoding='utf-8') as file:
    file.write(material_estudo)

print("�� Material de estudo criado!")
print("🧠 Contém conteúdo completo sobre Inteligência Artificial")
print("�� Arquivo 'material_ia.txt' salvo com sucesso!")

In [ ]:
# 👨‍🏫 CRIANDO O ASSISTENTE DE ESTUDOS

# Carregando material de estudo
loader = TextLoader('material_ia.txt', encoding='utf-8')
documentos = loader.load()

# Dividindo em pedaços menores
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)
textos_divididos = text_splitter.split_documents(documentos)

print(f"📄 Material dividido em {len(textos_divididos)} seções")

# Criando vector store para estudos
vectorstore_estudos = Chroma.from_documents(
    documents=textos_divididos,
    embedding=embeddings,
    collection_name="estudos_ia"
)

print("🧠 Vector store de estudos criado!")
print("📚 Base de conhecimento acadêmica pronta!")

In [ ]:
# 🎓 CRIANDO FUNÇÕES ESPECIALIZADAS DO ASSISTENTE

from langchain.schema import HumanMessage

def gerar_resumo(texto, llm):
    """Gera resumo do material de estudo"""
    prompt = f"""
    Crie um resumo conciso e didático do seguinte texto sobre IA:
    
    {texto}
    
    O resumo deve:
    - Ser fácil de entender
    - Destacar os pontos principais
    - Usar linguagem clara
    - Ter no máximo 200 palavras
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content
    except Exception as e:
        return f"Erro ao gerar resumo: {e}"

def criar_flashcards(texto, llm):
    """Cria flashcards para estudo"""
    prompt = f"""
    Crie 5 flashcards para estudo baseados no seguinte texto:
    
    {texto}
    
    Formato:
    Flashcard 1:
    Pergunta: [pergunta clara]
    Resposta: [resposta concisa]
    
    Flashcard 2:
    Pergunta: [pergunta clara]
    Resposta: [resposta concisa]
    
    E assim por diante...
    """
    
    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        return response.content
    except Exception as e:
        return f"Erro ao criar flashcards: {e}"

print("🎓 Funções especializadas criadas!")
print("📝 Pronto para gerar resumos e flashcards!")

In [ ]:
# 🧩 CRIANDO O SISTEMA COMPLETO DE ESTUDOS

class AssistenteEstudos:
    """Assistente de estudos inteligente"""
    
    def __init__(self, llm, vectorstore):
        self.llm = llm
        self.vectorstore = vectorstore
    
    def perguntar(self, pergunta):
        """Responde perguntas sobre o material de estudo"""
        try:
            # Buscando documentos relevantes
            docs = self.vectorstore.similarity_search(pergunta, k=3)
            contexto = "\n\n".join([doc.page_content for doc in docs])
            
            # Gerando resposta
            prompt = f"""
            Você é um professor especialista em IA. Responda à pergunta do aluno
            usando o seguinte material de estudo:
            
            MATERIAL: {contexto}
            
            PERGUNTA: {pergunta}
            
            Responda de forma didática e clara, como o Pedro Guth explicando.
            """
            
            response = self.llm.invoke([HumanMessage(content=prompt)])
            return response.content
            
        except Exception as e:
            return f"Erro: {e}"
    
    def gerar_resumo_tema(self, tema):
        """Gera resumo de um tema específico"""
        try:
            docs = self.vectorstore.similarity_search(tema, k=3)
            texto = "\n\n".join([doc.page_content for doc in docs])
            return gerar_resumo(texto, self.llm)
        except Exception as e:
            return f"Erro: {e}"
    
    def criar_flashcards_tema(self, tema):
        """Cria flashcards de um tema"""
        try:
            docs = self.vectorstore.similarity_search(tema, k=2)
            texto = "\n\n".join([doc.page_content for doc in docs])
            return criar_flashcards(texto, self.llm)
        except Exception as e:
            return f"Erro: {e}"

# Criando instância do assistente
assistente = AssistenteEstudos(llm, vectorstore_estudos)

print("🎓 Sistema completo de estudos criado!")
print("👨‍🏫 Tutor personalizado pronto para ajudar!")

### **Teste Rápido - Nosso Professor Digital em Ação!**

Vamos testar nosso assistente de estudos com algumas perguntas típicas:

In [ ]:
# 🧪 TESTE RÁPIDO: NOSSO PROFESSOR DIGITAL EM AÇÃO!

print("🎓 TESTANDO SISTEMA COMPLETO DE ESTUDOS")
print("=" * 60)

# Teste 1: Pergunta sobre Deep Learning
print("\n❓ PERGUNTA: O que é Deep Learning?")
resposta = assistente.perguntar("O que é Deep Learning e como funciona?")
print(f"��‍🏫 Professor: {resposta[:400]}...")

print("\n" + "-" * 40)

# Teste 2: Resumo sobre aplicações
print("\n�� RESUMO: Aplicações práticas de IA")
resumo = assistente.gerar_resumo_tema("aplicações práticas")
print(f"📋 Resumo:\n{resumo}")

print("\n" + "-" * 40)

# Teste 3: Flashcards sobre tipos de IA
print("\n🎴 FLASHCARDS: Tipos de Inteligência Artificial")
flashcards = assistente.criar_flashcards_tema("tipos de inteligência artificial")
print(f"🎴 Flashcards:\n{flashcards}")

print("\n" + "=" * 60)
print("�� Sistema de estudos funcionando perfeitamente!")

## **Aula 8.3: Projeto 3 - Sistema de Suporte Técnico**

---

### **O que vamos construir?**

Vamos criar um **sistema de suporte técnico inteligente** que:
- �� **Conhece todos os produtos e problemas**
- 🚨 **Diagnostica problemas automaticamente**
- 📋 **Gera tickets de suporte**
- 🎯 **Sugere soluções específicas**
- 📞 **Escala para humanos quando necessário**

É como ter um **técnico especializado 24h** que nunca se cansa! 🔧

**🖼️ Sugestão de imagem**: Um técnico digital diagnosticando problemas com ferramentas virtuais

### **Por que este projeto é valioso?**

**Problema real**: Empresas perdem tempo e dinheiro com suporte manual
**Solução**: Sistema automatizado que resolve 80% dos problemas

É como a diferença entre **uma fila de espera** e **resolução instantânea**! ⚡

In [ ]:
# 🔧 CRIANDO BASE DE CONHECIMENTO TÉCNICA

# Base de conhecimento sobre problemas técnicos
base_suporte = """
BASE DE CONHECIMENTO - SUPORTE TÉCNICO

PROBLEMAS COMUNS DE SMARTPHONES:

Problema: Bateria descarrega rápido
Causa: Apps em segundo plano, brilho alto, localização ativa
Solução: Fechar apps desnecessários, reduzir brilho, desativar localização
Prioridade: Baixa
Tempo estimado: 5 minutos

Problema: Tela não responde ao toque
Causa: Tela suja, película protetora, problema de hardware
Solução: Limpar tela, remover película, reiniciar dispositivo
Prioridade: Média
Tempo estimado: 10 minutos

Problema: Não conecta ao Wi-Fi
Causa: Senha incorreta, roteador desligado, problema de rede
Solução: Verificar senha, reiniciar roteador, testar em outra rede
Prioridade: Média
Tempo estimado: 15 minutos

PROBLEMAS COMUNS DE NOTEBOOKS:

Problema: Notebook muito lento
Causa: Muitos programas abertos, disco cheio, malware
Solução: Fechar programas, limpar disco, executar antivírus
Prioridade: Média
Tempo estimado: 30 minutos

Problema: Não liga
Causa: Bateria descarregada, carregador com defeito, problema de hardware
Solução: Conectar carregador, testar outro carregador, verificar tomada
Prioridade: Alta
Tempo estimado: 20 minutos

Problema: Tela azul
Causa: Driver desatualizado, problema de memória, sistema corrompido
Solução: Atualizar drivers, testar memória, restaurar sistema
Prioridade: Alta
Tempo estimado: 45 minutos

PROBLEMAS COMUNS DE ACESSÓRIOS:

Problema: Fones não conectam
Causa: Bluetooth desativado, dispositivo não pareado, bateria baixa
Solução: Ativar Bluetooth, parear dispositivo, carregar fones
Prioridade: Baixa
Tempo estimado: 10 minutos

Problema: Smartwatch não sincroniza
Causa: App desatualizado, Bluetooth desativado, problema de conta
Solução: Atualizar app, verificar Bluetooth, fazer login novamente
Prioridade: Baixa
Tempo estimado: 15 minutos

PROCEDIMENTOS DE ESCALAÇÃO:

Critérios para escalar para humano:
- Problema não resolvido após 3 tentativas
- Problema de hardware confirmado
- Cliente insatisfeito com solução
- Problema crítico de segurança

Informações necessárias para ticket:
- Nome do cliente
- Produto com problema
- Descrição detalhada do problema
- Tentativas de solução já realizadas
- Prioridade estimada
- Tempo estimado para resolução

CONTATOS DE SUPORTE:

Suporte Técnico Especializado:
- WhatsApp: (11) 99999-8888
- Email: suporte@techstore.com
- Horário: 24/7

Escalação para Supervisor:
- WhatsApp: (11) 99999-7777
- Email: supervisor@techstore.com
- Horário: 8h às 18h
"""

# Salvando base de suporte
with open('base_suporte.txt', 'w', encoding='utf-8') as file:
    file.write(base_suporte)

print("🔧 Base de conhecimento técnica criada!")
print("📋 Contém problemas comuns e soluções")
print("🎯 Pronto para diagnosticar e resolver problemas!")

In [ ]:
# 🚨 CRIANDO O SISTEMA DE SUPORTE TÉCNICO

# Carregando base de suporte
loader = TextLoader('base_suporte.txt', encoding='utf-8')
documentos = loader.load()

# Dividindo em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100
)
textos_divididos = text_splitter.split_documents(documentos)

print(f"📄 Base de suporte dividida em {len(textos_divididos)} seções")

# Criando vector store para suporte
vectorstore_suporte = Chroma.from_documents(
    documents=textos_divididos,
    embedding=embeddings,
    collection_name="suporte_tecnico"
)

print("�� Vector store de suporte criado!")
print("🚨 Sistema de diagnóstico ativo!")

In [ ]:
# 🎯 CRIANDO O SISTEMA COMPLETO DE SUPORTE

class SistemaSuporte:
    """Sistema de suporte técnico inteligente"""
    
    def __init__(self, llm, vectorstore):
        self.llm = llm
        self.vectorstore = vectorstore
        self.tentativas = 0
    
    def diagnosticar_problema(self, descricao):
        """Diagnostica o problema e sugere solução"""
        try:
            # Buscando soluções relevantes
            docs = self.vectorstore.similarity_search(descricao, k=3)
            contexto = "\n\n".join([doc.page_content for doc in docs])
            
            # Gerando diagnóstico
            prompt = f"""
            Você é um técnico especialista. Analise o problema e sugira uma solução:
            
            PROBLEMA: {descricao}
            
            BASE DE CONHECIMENTO: {contexto}
            
            Responda com:
            1. Diagnóstico do problema
            2. Solução passo a passo
            3. Tempo estimado
            4. Se precisa escalar para humano
            """
            
            response = self.llm.invoke([HumanMessage(content=prompt)])
            return response.content
            
        except Exception as e:
            return f"Erro no diagnóstico: {e}"
    
    def gerar_ticket(self, cliente, problema, solucao):
        """Gera ticket de suporte"""
        ticket = {
            "id": f"TICKET-{len(cliente) + len(problema)}",
            "cliente": cliente,
            "problema": problema,
            "solucao_sugerida": solucao,
            "status": "Aberto",
            "prioridade": "Média",
            "escalado": False
        }
        return ticket

# Criando instância do sistema
sistema_suporte = SistemaSuporte(llm, vectorstore_suporte)

print("�� Sistema de suporte técnico criado!")
print("🔧 Técnico digital pronto para diagnosticar problemas!")

### **Teste Rápido - Nosso Técnico Digital em Ação!**

Vamos testar nosso sistema de suporte com alguns problemas típicos:

In [ ]:
# 🧪 TESTE RÁPIDO: NOSSO TÉCNICO DIGITAL EM AÇÃO!

print("🚨 TESTANDO SISTEMA DE SUPORTE TÉCNICO")
print("=" * 60)

# Problemas para testar
problemas_teste = [
    "Meu iPhone está com a bateria descarregando muito rápido",
    "Meu notebook não está ligando",
    "Meus AirPods não conectam no celular",
    "Meu MacBook está muito lento"
]

for i, problema in enumerate(problemas_teste, 1):
    print(f"\n🔧 Problema {i}: {problema}")
    print("-" * 50)
    
    # Diagnosticando problema
    solucao = sistema_suporte.diagnosticar_problema(problema)
    print(f"🚨 Diagnóstico:\n{solucao[:300]}...")
    
    # Gerando ticket
    ticket = sistema_suporte.gerar_ticket(
        cliente=f"Cliente_{i}",
        problema=problema,
        solucao=solucao[:100]
    )
    
    print(f"📋 Ticket gerado: {ticket['id']}")
    print("-" * 50)

print("\n�� Sistema de suporte funcionando perfeitamente!")

### **Na Prática, Meu Consagrado!** 💪

**O que construímos nos Projetos Práticos:**

1. ✅ **Chatbot de E-commerce** - Vendedor digital 24/7
2. ✅ **Assistente de Estudos** - Professor particular digital
3. ✅ **Sistema de Suporte Técnico** - Técnico especializado digital
4. ✅ **Sistemas Completos** - Funcionais e prontos para uso
5. ✅ **Aplicação Real** - Problemas reais, soluções práticas

### **Vantagens dos Projetos**

- **Escalabilidade**: Podem atender milhares de usuários
- **Disponibilidade**: Funcionam 24/7 sem cansaço
- **Personalização**: Adaptam-se a cada usuário
- **Eficiência**: Automatizam tarefas complexas
- **Economia**: Reduzem custos operacionais
- **Qualidade**: Respostas consistentes e precisas

### **Próximos Passos**

**🖼️ Sugestão de imagem**: Screenshots dos projetos funcionando

**🎯 Próximo módulo**: Vamos aprender sobre **Deploy** - como colocar os projetos na rua!

**�� Resumo do Módulo 8**:
- ✅ Chatbot de E-commerce completo
- ✅ Assistente de Estudos inteligente
- ✅ Sistema de Suporte Técnico
- ✅ Sistemas funcionais e práticos
- ✅ Aplicação real dos conceitos

**🚀 Agora você tem projetos reais e funcionais!**

---

**💡 Dica do Pedro**: Os projetos práticos são a melhor forma de aprender RAG. Agora você pode adaptar esses sistemas para qualquer domínio!

**🚀 Próximo módulo**: Deploy - Colocando os projetos na rua! 🌐